In [53]:
import pandas as pd

## Scrape Web page content into dataframe

In [54]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
dfs = pd.read_html(url)

In [55]:
df = dfs[0]

In [60]:
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Dataframe Cleanup

#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [57]:
df = df.drop(df[df.Borough == 'Not assigned'].index)

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [58]:
df[df.Neighborhood == 'Not assigned']

,Postal Code,Borough,Neighborhood


--> No cell meets this criteria

In [59]:
print("Number of rows:", df.shape[0])

Number of rows: 103
